# Cleaning the Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

%config InlineBackend.figure_format = 'svg'
# %matplotlib inline

pd.options.display.max_rows = 20

In [2]:
# loading the compiled data
with open('pickles/may18_turnstiles.pickle', 'rb') as pik:
    data = pickle.load(pik)
    
# resetting incase something weird is going on
# df = pd.DataFrame(data) 

# handle the weird columns
df.columns = [column.strip() for column in df.columns]

# Generate a Weekday Column
df['DATETIME'] = pd.to_datetime(df.DATE +" " + df.TIME, format='%m/%d/%Y %H:%M:%S')
df['WEEKDAY_NUM'] = [dt.weekday() for dt in df['DATETIME']]

weekdays = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

df['WEEKDAY'] = [weekdays[day] for day in df['WEEKDAY_NUM']]

### Aside:

At this point we switched notebooks. We have put two notebooks together in as
logical a fashion as possible. I left the info about how to load it for my own
reference.

In [3]:
# ==============================
# loading the file with weekdays
# ==============================

FILE = r'pickles/with_weekdays.pickle'

with open(FILE, 'rb') as read_file:
    df = pickle.load(read_file)

df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,WEEKDAY_NUM,WEEKDAY
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,00:00:00,REGULAR,6625470,2245312,2018-05-19 00:00:00,5,Saturday
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,04:00:00,REGULAR,6625494,2245314,2018-05-19 04:00:00,5,Saturday
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,08:00:00,REGULAR,6625514,2245338,2018-05-19 08:00:00,5,Saturday
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,12:00:00,REGULAR,6625595,2245415,2018-05-19 12:00:00,5,Saturday
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,16:00:00,REGULAR,6625791,2245481,2018-05-19 16:00:00,5,Saturday


In [4]:
# Drop any duplicate entries, those being ones that occur at the same turnstile
# and the same time

turnstile = ["C/A", "UNIT", "SCP", "STATION"]

df.drop_duplicates(subset=turnstile+["DATETIME"], inplace=True)
df = df.sort_values(turnstile + ['DATE', 'WEEKDAY', 'TIME']) 
df[['PREV_TIME','PREV_ENTRIES','PREV_EXITS']] = \
                            (df.groupby(turnstile)['TIME','ENTRIES','EXITS']\
                            .transform(lambda x: x.shift(1)))

In [5]:
# Defines functions that allow us to cut off rows at an arbitrary cutoff.
# The arbitrary cutoff allows us to expirement and see which one makes the
# most sense.
#
# Negative values could still be useful. It could be that they were just
# just counting backwards. Therefore, we defined negatives to be positive.

def get_entry_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        counter = -counter
    if counter > max_counter: 
        return 0 
    return counter

def get_exit_counts(row, max_counter):
    counter = row["EXITS"] - row["PREV_EXITS"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        return 0
    return counter

# cut off any entries and exits that were greater than 4000
# took difference between consecutive counts

df["ENTRY_COUNTS"] = df.apply(get_entry_counts, axis=1, max_counter=4000) 
df["EXIT_COUNTS"] = df.apply(get_exit_counts, axis=1, max_counter=4000)

#  let's see how it did!
df[turnstile + ['DATETIME', 'ENTRY_COUNTS', 'EXIT_COUNTS']].head()

,C/A,UNIT,SCP,STATION,DATETIME,ENTRY_COUNTS,EXIT_COUNTS
0,A002,R051,02-00-00,59 ST,2018-04-28 00:00:00,NaN,NaN
1,A002,R051,02-00-00,59 ST,2018-04-28 04:00:00,17.0,1.0
2,A002,R051,02-00-00,59 ST,2018-04-28 08:00:00,16.0,33.0
3,A002,R051,02-00-00,59 ST,2018-04-28 12:00:00,81.0,92.0
4,A002,R051,02-00-00,59 ST,2018-04-28 16:00:00,214.0,60.0


In [6]:
# define total activity (entry + exit) field
df['TOTAL_ACTIVITY'] = df['ENTRY_COUNTS'] + df['EXIT_COUNTS']
df[turnstile + ['TOTAL_ACTIVITY']].head()

,C/A,UNIT,SCP,STATION,TOTAL_ACTIVITY
0,A002,R051,02-00-00,59 ST,NaN
1,A002,R051,02-00-00,59 ST,18.0
2,A002,R051,02-00-00,59 ST,49.0
3,A002,R051,02-00-00,59 ST,173.0
4,A002,R051,02-00-00,59 ST,274.0


In [7]:
# drop extraneous columns to save space

df.drop(['TIME', 'WEEKDAY_NUM', 'LINENAME', 'PREV_TIME'], axis= 1, inplace=True)
df.drop(['PREV_ENTRIES', 'PREV_EXITS'], axis=1, inplace=True)
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'DIVISION', 'DATE', 'DESC', 'ENTRIES',
       'EXITS', 'DATETIME', 'WEEKDAY', 'ENTRY_COUNTS', 'EXIT_COUNTS',
       'TOTAL_ACTIVITY'],
      dtype='object')

In [8]:
# How many rows are there?
len(df)

788761

In [9]:
# How many rows were affected by our 4000 total threshold?
len(df[df['ENTRY_COUNTS'] == 0])

117755

## Saving the data for later

In [10]:
with open('pickles/may_2018_with_counts.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)